In [ ]:
# general imports
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from time import perf_counter

In [ ]:
# experimental swarm modules
from swarm.ant_colony import AntColony
from swarm.ant_pheromones import BinsPheromones
from swarm.precedence import PrecedenceManager
from swarm.evolution_history import EvolutionHistory

from psplib.launcher import load_psplib_data, Launcher
from psplib.launcher import get_dict_of_optimized_data_structures

## Prepare data

In [ ]:
# loading data for the test
wg, contractors, work_estimator = load_psplib_data("psplib/j120_json/j1206_1.json")
ods = get_dict_of_optimized_data_structures(wg, contractors)

n_tasks = len(wg.nodes)
# to get metrics
launcher = Launcher(wg, contractors, work_estimator, ods)
# to convert to valid order
precedence_manager = PrecedenceManager.from_workgraph(wg)

In [ ]:
# for now, define fitness here
def fitness_function(priorities_array):
    activity_list = precedence_manager(priorities_array)
    makespan = launcher(activity_list)
    # -1 because we want higher = better (maybe?)
    return (-1) * makespan

## Create colony

In [ ]:
colony = AntColony(
    fitness_function=fitness_function,
    pheromones=BinsPheromones(
        n_tasks=n_tasks, 
        # use 5*5 structure for now
        n_bins=5, 
        n_sub_bins=5,
        # reproducible results 
        random=np.random.default_rng(seed=0)
    ),
    history=EvolutionHistory()
)

## Start the evolution

In [ ]:
# basic evolution
# can be interrupted, progress will be saved in history
colony.simple_evolution(n_generations=550, n_per_generation=25)

## Plot

In [ ]:
# info about evolution
stats = colony.history.get_stats()

# a super-simple baseline
baseline = max(
    fitness_function(np.arange(n_tasks)),
    fitness_function(np.arange(n_tasks)[::-1])
)
# for plotting, we need array with same length as others
baseline_line = baseline * np.ones(stats["n_generations"])

In [ ]:
# plot the history of evolution
fig = go.Figure([
    go.Scatter(y=stats["generation_best"], name="Gen Best", line_color="navy"),
    go.Scatter(y=stats["generation_mean"], name="Gen Mean", line_color="green"),
    go.Scatter(y=stats["best_so_far"], name="Best", line_color="black"),
    go.Scatter(y=baseline_line, name="Baseline", line_color="lightgrey")
])

fig.update_layout(height=500, width=1000, template="plotly_white")
fig.update_layout(
    xaxis_title="Generation",                   
    yaxis_title="Metric"
)
fig.update_layout(legend=dict(
    yanchor="middle", y=0.25,
    xanchor="center", x=0.75
))

fig.show()

## Testing integration

In [ ]:
from sampo.scheduler.base import Scheduler
from sampo.scheduler.resource.identity import IdentityResourceOptimizer
from sampo.pipeline import SchedulingPipeline

In [ ]:
class AntSheduler(Scheduler):

    def __init__(self, resource_optimizer, work_estimator):
        super().__init__(
            scheduler_type="AntScheduler[Experimental]",
            resource_optimizer=resource_optimizer,
            work_estimator=work_estimator
        )
    
    def __str__(self):
        return "AntScheduler[Experimental]"

    def schedule_with_cache(self, wg, contractors, *args, **kwargs):
        ods = get_dict_of_optimized_data_structures(wg, contractors)
        
        n_tasks = len(wg.nodes)
        launcher = Launcher(wg, contractors, self.work_estimator, ods)
        precedence_manager = PrecedenceManager.from_workgraph(wg)

        def fitness_function(priorities_array):
            activity_list = precedence_manager(priorities_array)
            makespan = launcher(activity_list)
            return (-1) * makespan

        colony = AntColony(
            fitness_function=fitness_function,
            pheromones=BinsPheromones(
                n_tasks=n_tasks, 
                n_bins=5, 
                n_sub_bins=5,
                random=np.random.default_rng(seed=0)
            ),
            history=EvolutionHistory()
        )
        colony.simple_evolution(n_generations=10, n_per_generation=25)

        best_solution = colony.history.best_solution
        best_activity_list = precedence_manager(best_solution)
        best_schedule = launcher.get_schedule(best_activity_list)
        return [best_schedule]

In [ ]:
wg, contractors, work_estimator = load_psplib_data("psplib/j120_json/j1206_1.json")

In [ ]:
ant_sheduler = AntSheduler(
    work_estimator=work_estimator,
    resource_optimizer=IdentityResourceOptimizer()
)

In [ ]:
schedule = (
    SchedulingPipeline
    .create()
    .wg(wg)
    .contractors(contractors)
    .schedule(ant_sheduler)
    .finish()
)[0]